In [5]:
import numpy as np
import pandas as pd
import pystan
from scipy.stats import norm, multivariate_normal, invwishart, invgamma, bernoulli
from scipy.special import expit

import datetime
import sys
import os

from codebase.plot import * 
from codebase.file_utils import save_obj, load_obj

%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
np.random.seed(121)
nsim_data = 200
J = 6
K = 2
beta = np.array([[1,0], [-2, 2],[-1,-1],[2,1], [3,-1], [1,-1]], dtype=float)
mu = np.array([1,2,.3,-.8, 1, -1.4])
Sigma = np.eye(J)*0.04
ee = multivariate_normal.rvs(mean = np.zeros(J), cov = Sigma, size=nsim_data)
zz = multivariate_normal.rvs(mean = np.zeros(K), cov=np.eye(K), size=nsim_data)
yy = mu + zz@beta.T + ee
DD = bernoulli.rvs(p=expit(yy)); DD

# err = np.abs((beta@beta.T + Sigma) - np.cov(yy.T))

array([[1, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 0],
       [0, 1, 1, 0, 0, 0],
       ...,
       [0, 1, 0, 0, 1, 0],
       [0, 1, 1, 0, 0, 0],
       [1, 0, 1, 0, 1, 0]])

In [7]:
data = dict()
data['N'] = nsim_data
data['K'] = K
data['J'] = J
data['Sigma'] = Sigma
data['y'] = yy
data['D'] = DD
data['beta'] = beta
data['u'] = ee
data['mu'] = mu
data['Omega'] = data['beta']@data['beta'].T + data['Sigma']

In [8]:
stan_data = dict(N = data['N'], K = data['K'], J = data['J'], DD = data['D'])


In [9]:
with open('./codebase/stan_code/model6.stan', 'r') as file:
    model_code = file.read()


In [11]:
sm = pystan.StanModel(model_code=model_code, verbose=True)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_6e64cfa7ed64b74881d536c069d95d79 NOW.
INFO:pystan:OS: darwin, Python: 3.6.4 | packaged by conda-forge | (default, Dec 23 2017, 16:54:01) 
[GCC 4.2.1 Compatible Apple LLVM 6.1.0 (clang-602.0.53)], Cython 0.29.6


Compiling /var/folders/x4/9t2lxpnx30q0xbrfyc3q9ghr0000gn/T/tmpsj8fqiyu/stanfit4anon_model_6e64cfa7ed64b74881d536c069d95d79_647410158189377304.pyx because it changed.
[1/1] Cythonizing /var/folders/x4/9t2lxpnx30q0xbrfyc3q9ghr0000gn/T/tmpsj8fqiyu/stanfit4anon_model_6e64cfa7ed64b74881d536c069d95d79_647410158189377304.pyx
building 'stanfit4anon_model_6e64cfa7ed64b74881d536c069d95d79_647410158189377304' extension
creating /var/folders/x4/9t2lxpnx30q0xbrfyc3q9ghr0000gn/T/tmpsj8fqiyu/var
creating /var/folders/x4/9t2lxpnx30q0xbrfyc3q9ghr0000gn/T/tmpsj8fqiyu/var/folders
creating /var/folders/x4/9t2lxpnx30q0xbrfyc3q9ghr0000gn/T/tmpsj8fqiyu/var/folders/x4
creating /var/folders/x4/9t2lxpnx30q0xbrfyc3q9ghr0000gn/T/tmpsj8fqiyu/var/folders/x4/9t2lxpnx30q0xbrfyc3q9ghr0000gn
creating /var/folders/x4/9t2lxpnx30q0xbrfyc3q9ghr0000gn/T/tmpsj8fqiyu/var/folders/x4/9t2lxpnx30q0xbrfyc3q9ghr0000gn/T
creating /var/folders/x4/9t2lxpnx30q0xbrfyc3q9ghr0000gn/T/tmpsj8fqiyu/var/folders/x4/9t2lxpnx30q0xbrfyc3q9ghr0000

In [14]:
nowstr = datetime.datetime.now().strftime('%Y%m%d_%H%M%S_') # ISO 8601 format
nowstr

'20190319_195434_'

In [20]:
task_id = 'NN1'
log_dir =  "./log/"+nowstr+"%s/" % task_id
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [32]:
num_chains = 1
num_iter = 100

In [22]:
fit = sm.sampling(data=stan_data, iter=num_iter, chains=number_of_chains)


To run all diagnostics call pystan.check_hmc_diagnostics(fit)


In [25]:
save_obj(sm, 'sm', log_dir)
save_obj(fit, 'fit', log_dir)


/Users/Konstantin/myPhd/factor-analysis/src/codebase/file_utils.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


In [36]:
post_samples = fit.extract(permuted=False, pars=['Omega'])  # return a dictionary of arrays
post_samples['Omega'].shape
ps=dict()
for i in range(num_chains):
    ps[i] = post_samples['Omega'][:,i,:,:]

In [37]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['J']):
    for j in range(data['J']):
            plots.append(plot_trace(ps[0][:,i,j],
                     true_value=data['Omega'][i,j],
                     title = 'Posterior distribution for Omega(%s,%s)'%(i,j)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_0_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_0_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_0_comma_2_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_0_comma_3_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_0_comma_4_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_0_comma_5_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_1_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_1_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_1_comma_2_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_1_comma_3_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_1_comma_4_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_1_comma_5_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_2_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_2_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_2_comma_2_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_2_comma_3_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_2_comma_4_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_2_comma_5_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_3_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_3_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_3_comma_2_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_Omega_left_parenthesis_3_comma_3_right_parenthesis :Overlay
      .C

In [40]:
post_samples = fit.extract(permuted=False, pars=['mu'])  # return a dictionary of arrays
post_samples['mu'].shape
ps=dict()
for i in range(num_chains):
    ps[i] = post_samples['mu'][:,i,:]

In [41]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['J']):
            plots.append(plot_trace(ps[0][:,i],
                     true_value=data['mu'][i],
                     title = 'Posterior distribution for mu(%s)'%(i)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Overlay.Posterior_distribution_for_mu_left_parenthesis_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_mu_left_parenthesis_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_mu_left_parenthesis_2_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_mu_left_parenthesis_3_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_mu_left_parenthesis_4_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_mu_left_parenthesis_5_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)